In [ ]:
import pandas as pd
import sqlite3
db_path = '../database/dc_hoteldb.db3'

In [ ]:
query = """
    SELECT  
		id,
		COALESCE(location, 'Unknown') AS location, 
		CASE
			WHEN total_rooms is NULL THEN 100
			WHEN total_rooms <1 OR total_rooms>400 THEN 100
			ELSE total_rooms
		END as total_rooms,
		
		CASE
			WHEN staff_count is NULL THEN (COALESCE(total_rooms,100)*1.5)::INTEGER
			ELSE staff_count
		END as staff_count, 
		
		CASE
			WHEN opening_date is NULL THEN 2023
			WHEN opening_date = '-' THEN 2023
			WHEN opening_date::INTEGER <2000 OR opening_date::INTEGER>2023 THEN 2023
			ELSE opening_date::INTEGER
		END AS opening_date,
	 
		CASE
       	 	WHEN target_guests IS NULL THEN 'Leisure'
        	WHEN LOWER(target_guests) LIKE 'b%' THEN 'Business'
        	ELSE target_guests 
		END AS target_guests
		FROM branch;

    """
df = pd.read_sql_query(query, f"sqlite:///{db_path}")
df

In [ ]:
query = """

        SELECT 
            service_id,
            branch_id, 
            ROUND(AVG(CAST(time_taken AS INTEGER)),2) as avg_time_taken,
            ROUND(MAX(CASE(time_taken AS INTEGER)), 2) as max_time_taken
            
            FROM public.request
            GROUP BY service_id, branch_id
            ORDER BY service_id, branch_id;

        """

df = pd.read_sql_query(query, f"sqlite:///{db_path}")
df

In [ ]:
query = """
    SELECT 
            
        s.description,
        b.id, 
        b.location,
        r.id AS request_id,	
        r.rating
        
        FROM request AS r
        INNER JOIN branch b
        ON b.id = r.branch_id
        INNER JOIN service as s
        ON r.service_id = s.id
        WHERE b.location in ('EMEA', 'LATAM') 
        AND (s.description ILIKE '%Meal%' OR s.description ILIKE '%Laundry%')
        ORDER by location, description;
    """

df = pd.read_sql_query(query, f"sqlite:///{db_path}")
df

In [ ]:
query = """
SELECT 
	service_id,
	branch_id,
	ROUND(AVG(rating),2) as avg_rating
    FROM request
    GROUP BY service_id, branch_id
    HAVING AVG(rating)<4.5;

"""

df = pd.read_sql_query(query, f"sqlite:///{db_path}")
df